In [1]:
lines_list = []
with open('E:/ctc/mysql.sql',  "r")  as  fileHandler:
    while line := fileHandler.readline():
        lines_list.append(line.strip())

required_txt = "`ID` bigint(20)"
id_not_auto_increment = []
id_auto_increment = []
for idx, line in enumerate(lines_list):
    if required_txt in line:
        if "AUTO_INCREMENT" in line:
            id_auto_increment.append(lines_list[idx-1].split("`")[1])
        else:
            id_not_auto_increment.append(lines_list[idx-1].split("`")[1])

postsql_list = []
with open('E:/ctc/pg_sql.sql',  "r")  as  fileHandler:
    while line := fileHandler.readline():
        postsql_list.append(line.strip())
new_list = []
change_flag = 0
for post_line in postsql_list:
    
    if "CREATE TABLE" in post_line:
        table = post_line.split('"')[-2]
        change_flag = 1 if table in id_auto_increment else 0
        # post_line = post_line.replace('"', '')
        new_list.append(
            post_line.replace("CREATE TABLE", "CREATE TABLE IF NOT EXISTS")
        )
        continue

    if change_flag == 1:
        post_line = post_line.replace("numeric(20)", "bigserial")
        change_flag = 0

    if "float8" in post_line:
        post_line = post_line.replace("float8", "numeric(20, 6)")
    if "CREATE UNIQUE INDEX" in post_line:
        index_name = post_line.split('"')[1]
        table = post_line.split('"')[-2]
        post_line = post_line.replace(index_name, f"{table}_{index_name}", 1)
    if "CREATE INDEX" in post_line:
        index_name = post_line.split('"')[1]
        table = post_line.split('"')[-2]
        post_line = post_line.replace(index_name, f"{table}_{index_name}", 1)
    if "timestamp" in post_line:
        post_line = post_line.replace("timestamp", "timestamp default now()")

    if 'PRIMARY KEY ("id")' in post_line:
        table = post_line.split(' ')[2]
        index_name = table.split('"')[-2] + "_id"
        post_line = f'CREATE INDEX "{index_name}" on {table} using btree (id);'
    # post_line = post_line.replace('"', '')
    new_list.append(post_line)

with open('E:/ctc/new.txt',  "w") as fileHandler:
    fileHandler.write('\n'.join(new_list))

In [10]:
from quant_utils.db_conn import DB_CONN_PG_LOCAL, DB_CONN_JJTG_DATA

In [11]:
table_query = """
SELECT
  a.table_schema,
  a.table_name,
  round(sum(DATA_LENGTH / 1024 / 1024/1024) + sum(INDEX_LENGTH / 1024 / 1024/1024), 2) total_size,
  round(sum(DATA_LENGTH / 1024 / 1024/1024), 2) AS data_size,
  round(sum(INDEX_LENGTH / 1024 / 1024/1024), 2) AS index_size 
FROM
  information_schema.TABLES a 
WHERE
  a.table_schema = 'chentiancheng' 
  and a.table_type = 'BASE TABLE'
GROUP BY
  a.table_schema,
  a.table_name 
ORDER BY
  data_size ASC
"""
df = DB_CONN_JJTG_DATA.exec_query(table_query)


exec_query耗时:0.0130 s


In [12]:
df = df.query('data_size < 2')

In [13]:
error_dict = {}
for index, row in df.iterrows():
    table = row['TABLE_NAME']
    # unique_indexs = DB_CONN_JJTG_DATA.get_db_table_unque_index(table)
    # if not unique_indexs:
    #     print (table)
    query = f"select * from {table}"
    result = DB_CONN_JJTG_DATA.exec_query(query)
    try:
        DB_CONN_PG_LOCAL.upsert(result, row['TABLE_NAME'])
        print("=="*35)
    except Exception as e:
        error_dict[row['TABLE_NAME']] = e